In [1]:
import pandas as pd
import numpy as np
import json
import requests


In [3]:
# Parse Game Data

data = []

for game in np.arange(1,1272):
    
    print('precent complete:', game/1271*100,' game: ', game)

    game_str = str(game).zfill(4)
    game_event_data = requests.get('https://statsapi.web.nhl.com/api/v1/game/201902' + game_str + '/feed/live')

    json_data_event = json.loads(game_event_data.text)
    json_data_event
    
    one_game = {}

    
    ###Get team stats and game results###
    one_game.update({"game_id" : json_data_event['gameData']['game']['pk'],
                     "game_type" : json_data_event['gameData']['game']['type'],
                     "game_season" : json_data_event['gameData']['game']['season'],
                     "game_date" : json_data_event['gameData']['datetime']['dateTime']})

    one_game.update({"away_team": json_data_event['liveData']['boxscore']['teams']['away']['team']['name'],
                    "away_team_id" : json_data_event['liveData']['boxscore']['teams']['away']['team']['id']})

    away_stats = json_data_event['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']
    one_game.update({'away_' + k: v for k, v in away_stats.items()})


    one_game.update({"home_team": json_data_event['liveData']['boxscore']['teams']['home']['team']['name'],
                    "home_team_id" : json_data_event['liveData']['boxscore']['teams']['home']['team']['id']})

    away_stats = json_data_event['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']
    one_game.update({'home_' + k: v for k, v in away_stats.items()})
    
    one_game.update({"game_end_type" : json_data_event['liveData']['linescore']['currentPeriod'] })
    
    ###Get Player Info (Goalie Save Percentage Specifically at first)###
    
    home_player_data = json_data_event['liveData']['boxscore']['teams']['home']['players']

    #Saves, #Shots Against
    cnt = [ 0, 0 ]
    for player in home_player_data:
        position = home_player_data[player]['position']['code']

        if position == 'G':
            #print(home_player_data[player])
            saves = home_player_data[player]['stats']['goalieStats']['saves']
            shots = home_player_data[player]['stats']['goalieStats']['shots']

            cnt[0] = cnt[0] + saves
            cnt[1] = cnt[1] + shots
            
            if cnt[1] > 0:
                home_SV_Pct = cnt[0] / cnt[1]

    away_player_data = json_data_event['liveData']['boxscore']['teams']['away']['players']

    #Saves, #Shots Against
    cnt = [ 0, 0 ]

    for player in away_player_data:
        position = away_player_data[player]['position']['code']

        if position == 'G':
            #print(away_player_data[player])
            saves = away_player_data[player]['stats']['goalieStats']['saves']
            shots = away_player_data[player]['stats']['goalieStats']['shots']

            cnt[0] = cnt[0] + saves
            cnt[1] = cnt[1] + shots
            
            if cnt[1] > 0:
                away_SV_Pct = cnt[0] / cnt[1]

    #print('Home SV%: ',home_SV_Pct,'Away SV%: ',away_SV_Pct)
    #print('Home SH%: ',1 - away_SV_Pct,'Away SH%: ',1 - home_SV_Pct)
    
    one_game.update({"home_SV_Pct" : home_SV_Pct,
                     "away_SV_Pct" : away_SV_Pct,
                     "home_SH_Pct" : 1 - away_SV_Pct,
                     "away_SH_Pct" : 1 - home_SV_Pct})
    
    ####Get Corsi For Percentage####
    
    cnt = [ 0, 0, 0, 0, 0, 0 ]
    for t in json_data_event['liveData']['plays']['allPlays']:

        event = t['result']['event']

        #Home: Shot, missed shot, blocked shot,Away: Shot, missed shot, blocked shot

        if "Shot" in event: 

            #description = t['result']['description']
            #print(event,description)
            team = t['team']

            #print(team['id'])

            if team['id'] == one_game['home_team_id']:
                    if "Missed Shot" in event:
                        cnt[1] = cnt[1] + 1
                    elif "Blocked Shot" in event:    
                        cnt[5] = cnt[5] + 1
                    else:
                        cnt[0] = cnt[0] + 1

            if team['id'] == one_game['away_team_id']:
                    if "Missed Shot" in event:
                        cnt[4] = cnt[4] + 1
                    elif "Blocked Shot" in event:    
                        cnt[2] = cnt[2] + 1
                    else:
                        cnt[3] = cnt[3] + 1

    #print(cnt)

    #Corsi For % (CF%) = CF / (CF + CA)
    
    try:
        home_CF_Pct = sum(cnt[0:2]) / ( sum(cnt[0:2]) + sum(cnt[3:5]) )
        away_CF_Pct = sum(cnt[3:5]) / ( sum(cnt[0:2]) + sum(cnt[3:5]) )
    except:
        home_CF_Pct = 0
        away_CF_Pct = 0
        
    #print('Home CF%: ',home_CF_Pct,'Away CF%: ',away_CF_Pct)    

    one_game.update({"home_CF_Pct" : home_CF_Pct,
                     "away_CF_Pct" : away_CF_Pct})
    
    ## Win Label ##
    
    home_win_label = one_game['home_goals'] > one_game['away_goals']
    
    one_game.update({"home_win" : home_win_label})
    
    ###Get/Calculate Event Data other Fancy Stats xG ?###
    
    
    
    ### Place data in a list###
    data.append(list(one_game.values()))

###Put data in DataFrame###

game_data_df = pd.DataFrame(data,columns=list(one_game.keys()))

precent complete: 0.07867820613690008  game:  1
precent complete: 0.15735641227380015  game:  2
precent complete: 0.23603461841070023  game:  3
precent complete: 0.3147128245476003  game:  4
precent complete: 0.3933910306845004  game:  5
precent complete: 0.47206923682140045  game:  6
precent complete: 0.5507474429583006  game:  7
precent complete: 0.6294256490952006  game:  8
precent complete: 0.7081038552321007  game:  9
precent complete: 0.7867820613690008  game:  10
precent complete: 0.865460267505901  game:  11
precent complete: 0.9441384736428009  game:  12
precent complete: 1.0228166797797011  game:  13
precent complete: 1.1014948859166012  game:  14
precent complete: 1.1801730920535012  game:  15
precent complete: 1.2588512981904012  game:  16
precent complete: 1.3375295043273014  game:  17
precent complete: 1.4162077104642015  game:  18
precent complete: 1.4948859166011015  game:  19
precent complete: 1.5735641227380015  game:  20
precent complete: 1.6522423288749015  game:  2

precent complete: 13.611329661683714  game:  173
precent complete: 13.690007867820613  game:  174
precent complete: 13.768686073957515  game:  175
precent complete: 13.847364280094416  game:  176
precent complete: 13.926042486231314  game:  177
precent complete: 14.004720692368213  game:  178
precent complete: 14.083398898505115  game:  179
precent complete: 14.162077104642016  game:  180
precent complete: 14.240755310778914  game:  181
precent complete: 14.319433516915813  game:  182
precent complete: 14.398111723052715  game:  183
precent complete: 14.476789929189614  game:  184
precent complete: 14.555468135326516  game:  185
precent complete: 14.634146341463413  game:  186
precent complete: 14.712824547600315  game:  187
precent complete: 14.791502753737214  game:  188
precent complete: 14.870180959874116  game:  189
precent complete: 14.948859166011013  game:  190
precent complete: 15.027537372147915  game:  191
precent complete: 15.106215578284814  game:  192
precent complete: 15

precent complete: 26.90794649881983  game:  342
precent complete: 26.986624704956725  game:  343
precent complete: 27.065302911093625  game:  344
precent complete: 27.143981117230524  game:  345
precent complete: 27.222659323367427  game:  346
precent complete: 27.301337529504327  game:  347
precent complete: 27.380015735641226  game:  348
precent complete: 27.45869394177813  game:  349
precent complete: 27.53737214791503  game:  350
precent complete: 27.61605035405193  game:  351
precent complete: 27.694728560188832  game:  352
precent complete: 27.773406766325724  game:  353
precent complete: 27.852084972462627  game:  354
precent complete: 27.930763178599527  game:  355
precent complete: 28.009441384736427  game:  356
precent complete: 28.08811959087333  game:  357
precent complete: 28.16679779701023  game:  358
precent complete: 28.24547600314713  game:  359
precent complete: 28.324154209284032  game:  360
precent complete: 28.40283241542093  game:  361
precent complete: 28.4815106

precent complete: 40.28324154209284  game:  512
precent complete: 40.36191974822974  game:  513
precent complete: 40.440597954366645  game:  514
precent complete: 40.519276160503544  game:  515
precent complete: 40.597954366640444  game:  516
precent complete: 40.676632572777336  game:  517
precent complete: 40.755310778914236  game:  518
precent complete: 40.83398898505114  game:  519
precent complete: 40.91266719118804  game:  520
precent complete: 40.99134539732494  game:  521
precent complete: 41.07002360346184  game:  522
precent complete: 41.14870180959874  game:  523
precent complete: 41.22738001573564  game:  524
precent complete: 41.30605822187255  game:  525
precent complete: 41.38473642800944  game:  526
precent complete: 41.46341463414634  game:  527
precent complete: 41.54209284028324  game:  528
precent complete: 41.62077104642014  game:  529
precent complete: 41.699449252557045  game:  530
precent complete: 41.778127458693945  game:  531
precent complete: 41.856805664830

precent complete: 53.65853658536586  game:  682
precent complete: 53.73721479150275  game:  683
precent complete: 53.81589299763966  game:  684
precent complete: 53.89457120377655  game:  685
precent complete: 53.97324940991345  game:  686
precent complete: 54.05192761605036  game:  687
precent complete: 54.13060582218725  game:  688
precent complete: 54.209284028324156  game:  689
precent complete: 54.28796223446105  game:  690
precent complete: 54.366640440597955  game:  691
precent complete: 54.445318646734854  game:  692
precent complete: 54.52399685287176  game:  693
precent complete: 54.602675059008654  game:  694
precent complete: 54.681353265145546  game:  695
precent complete: 54.76003147128245  game:  696
precent complete: 54.83870967741935  game:  697
precent complete: 54.91738788355626  game:  698
precent complete: 54.99606608969315  game:  699
precent complete: 55.07474429583006  game:  700
precent complete: 55.15342250196695  game:  701
precent complete: 55.23210070810386

precent complete: 67.03383162863886  game:  852
precent complete: 67.11250983477576  game:  853
precent complete: 67.19118804091266  game:  854
precent complete: 67.26986624704956  game:  855
precent complete: 67.34854445318646  game:  856
precent complete: 67.42722265932338  game:  857
precent complete: 67.50590086546026  game:  858
precent complete: 67.58457907159718  game:  859
precent complete: 67.66325727773406  game:  860
precent complete: 67.74193548387096  game:  861
precent complete: 67.82061369000787  game:  862
precent complete: 67.89929189614476  game:  863
precent complete: 67.97797010228167  game:  864
precent complete: 68.05664830841856  game:  865
precent complete: 68.13532651455547  game:  866
precent complete: 68.21400472069237  game:  867
precent complete: 68.29268292682927  game:  868
precent complete: 68.37136113296617  game:  869
precent complete: 68.45003933910306  game:  870
precent complete: 68.52871754523997  game:  871
precent complete: 68.60739575137687  gam

precent complete: 80.48780487804879  game:  1023
precent complete: 80.56648308418568  game:  1024
precent complete: 80.64516129032258  game:  1025
precent complete: 80.72383949645948  game:  1026
precent complete: 80.80251770259638  game:  1027
precent complete: 80.88119590873329  game:  1028
precent complete: 80.95987411487017  game:  1029
precent complete: 81.03855232100709  game:  1030
precent complete: 81.11723052714397  game:  1031
precent complete: 81.19590873328089  game:  1032
precent complete: 81.27458693941779  game:  1033
precent complete: 81.35326514555467  game:  1034
precent complete: 81.43194335169159  game:  1035
precent complete: 81.51062155782847  game:  1036
precent complete: 81.58929976396539  game:  1037
precent complete: 81.66797797010229  game:  1038
precent complete: 81.74665617623918  game:  1039
precent complete: 81.82533438237608  game:  1040
precent complete: 81.90401258851298  game:  1041
precent complete: 81.98269079464988  game:  1042
precent complete: 82

precent complete: 93.70574350904799  game:  1191
precent complete: 93.78442171518489  game:  1192
precent complete: 93.86309992132179  game:  1193
precent complete: 93.94177812745869  game:  1194
precent complete: 94.02045633359559  game:  1195
precent complete: 94.0991345397325  game:  1196
precent complete: 94.17781274586939  game:  1197
precent complete: 94.25649095200629  game:  1198
precent complete: 94.3351691581432  game:  1199
precent complete: 94.41384736428009  game:  1200
precent complete: 94.492525570417  game:  1201
precent complete: 94.57120377655389  game:  1202
precent complete: 94.6498819826908  game:  1203
precent complete: 94.72856018882769  game:  1204
precent complete: 94.8072383949646  game:  1205
precent complete: 94.8859166011015  game:  1206
precent complete: 94.96459480723838  game:  1207
precent complete: 95.0432730133753  game:  1208
precent complete: 95.1219512195122  game:  1209
precent complete: 95.2006294256491  game:  1210
precent complete: 95.279307631

In [4]:
game_data_df.to_csv('2019_game_data.csv',index=False)

In [ ]:
# Seperate by team and create 20 game rolling average and 